Import of libraries

In [813]:
import nltk

In [814]:
import re
from sklearn.metrics import confusion_matrix

In [815]:
import pandas as pd
import numpy as np

In [816]:
import csv

In [817]:
import os
import matplotlib.pyplot as plt

In [818]:

import sys
import os
import time

In [819]:
import random

In [820]:

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
from sklearn.metrics import classification_report

Setting working directory

In [821]:
print os.getcwd();

/Users/imacair/Desktop/Products3


In [822]:
os.chdir("/Users/imacair/Desktop/Products3/")

Read the file

In [823]:
data = pd.read_csv('Final_Manual_0805.csv', encoding= "latin-1",delimiter=',',low_memory=False)
#data



In [824]:
data.head(10)

,key,created_time,From.User,message,Id,sentiment,sarcastic,context,topic
0,ankylosing spondylitis,2017-03-14,Exeter EM education,#llbite ankylosing spondylitis - have a very l...,1623,2,0,0,"spinal, injury"
1,ankylosing spondylitis,2017-03-13,Catarina Marques,#as 3 things people with ankylosing spondyliti...,1624,3,0,0,ankylosing spondylitis
2,ankylosing spondylitis,2017-03-13,MD Magazine,watch physicians discuss characteristics of ps...,1625,3,0,0,ankylosing spondylitis
3,ankylosing spondylitis,2017-03-13,Duffitness,i dont know if its the medications or pinching...,1626,1,1,0,"ankylosing spondylitis, pain, medication"
4,ankylosing spondylitis,2017-03-13,PPM Journal,#imaginedragons singer @danreynolds opens up a...,1627,2,0,0,"ankylosing spondylitis, struggle, musician"
5,ankylosing spondylitis,2017-03-12,chenjun884899,chinese medicine cure ankylosing spondylitis c...,1628,3,0,0,"ankylosing spondylitis, cure, medication"
6,ankylosing spondylitis,2017-03-11,Vidalux,pain relief for patients with rheumatoid arthr...,1629,3,0,0,"ankylosing spondylitis, relief"
7,ankylosing spondylitis,2017-03-11,SpineUniverse,suffering from #ankylosingspondylitis? check o...,1630,3,0,0,"ankylosing spondylitis, medication"
8,ankylosing spondylitis,2017-03-11,AdderiyCarl,herself goodwill newsletter so that ankylosing...,1631,2,0,0,"ankylosing spondylitis, newsletter"
9,ankylosing spondylitis,2017-03-11,Gracie Bagosy-Young,how they did it: 5 people with ankylosing spon...,1632,3,0,0,"ankylosing spondylitis, dreams"


In [825]:
#Converts text into ASCII

In [826]:
data.message = data.message.str.encode('ascii','replace')

In [827]:
data.sentiment = data.sentiment.str.encode('utf-8','replace')

In [830]:
#number of elements
len(data)

965

In [831]:
#In case if you need a sample of data
#sample_data= data.sample(n=64000)

In [832]:
sample_data= data

In [833]:
data_t=sample_data["message"]

In [834]:
len(data_t)

965

In [835]:
#lowercase before abbriviation translation
data_t = data_t.str.lower()

In [836]:
data_s=sample_data["sentiment"]

Abbriviation translation raplacing elements from file 

In [839]:
with open('abbrev.csv', mode='r') as infile:
    reader = csv.reader(infile)
    replacement = {rows[0].lower():rows[1].lower() for rows in reader              
                  }

In [840]:
#replacement

In [841]:
#replacement = {
##'r':'are',
#'y':'why',
#'u':'you'}

In [842]:
##How in works
s1 = 'y r u l8'

s2 = ' '.join([replacement.get(w, w) for w in s1.split()])
s2

'why? -or- yes are you l8'

In [843]:
result = pd.DataFrame()
result = data_t

In [844]:
for i in range(len(result)):
    data_t.values[i]=' '.join([replacement.get(w, w) for w in data_t.values[i].split()])

In [845]:
data_t.head(10)

0    #llbite ankylosing spondylitis - have (in date...
1    #as 3 things people with ankylosing spondyliti...
2    working group on the assessment of toxic chemi...
3    i dont know if intense text sex the medication...
4    #imaginedragons singer @danreynolds opens up a...
5    chinese medicine cure ankylosing spondylitis c...
6    pain relief for patients with rheumatoid arthr...
7    suffering from #ankylosingspondylitis? check o...
8    herself goodwill newsletter significant other ...
9    how they did it: 5 people with ankylosing spon...
Name: message, dtype: object

In [846]:
#lowercase
data_t = data_t.str.lower()
#Remove urls
data_t= data_t.str.replace(r'(http.*) |(http.*)$|\n', "",)
#Remove twitter handles
data_t = data_t.str.replace(r"@\\w+", "")
#remove htmls
data_t = data_t.str.replace(r'<.*?>', "")
#Remove citations
data_t = data_t.str.replace(r'@[a-zA-Z0-9]*', "")

#remove _
#data_t = data_t.str.replace(r'\_+',"")

In [847]:
data_t.head(10)

0    #llbite ankylosing spondylitis - have (in date...
1    #as 3 things people with ankylosing spondyliti...
2    working group on the assessment of toxic chemi...
3    i dont know if intense text sex the medication...
4    #imaginedragons singer  opens up about his str...
5    chinese medicine cure ankylosing spondylitis c...
6    pain relief for patients with rheumatoid arthr...
7    suffering from #ankylosingspondylitis? check o...
8    herself goodwill newsletter significant other ...
9    how they did it: 5 people with ankylosing spon...
Name: message, dtype: object

In [848]:
from textblob import TextBlob

In [849]:
data_t.head(10)

0    #llbite ankylosing spondylitis - have (in date...
1    #as 3 things people with ankylosing spondyliti...
2    working group on the assessment of toxic chemi...
3    i dont know if intense text sex the medication...
4    #imaginedragons singer  opens up about his str...
5    chinese medicine cure ankylosing spondylitis c...
6    pain relief for patients with rheumatoid arthr...
7    suffering from #ankylosingspondylitis? check o...
8    herself goodwill newsletter significant other ...
9    how they did it: 5 people with ankylosing spon...
Name: message, dtype: object

In [855]:
#Creating a column for polarity
res= pd.DataFrame(np.random.randn(len(data_t), 1),columns = {'polarity'} )

In [856]:
#Creating a column for subjectivity
res2= pd.DataFrame(np.random.randn(len(data_t), 1),columns = {'subjectivity'} )

Filling the columns with values

In [858]:
for i in range(len(data_t)):
    testimonial = TextBlob(data_t.values[i])
    res.values[i]= testimonial.sentiment.polarity

In [859]:
for i in range(len(data_t)):
    testimonial = TextBlob(data_t.values[i])
    res2.values[i]= testimonial.sentiment.subjectivity


Merging with initial df

In [863]:
final=pd.concat([data_t, res], axis=1)

In [864]:
final=pd.concat([final, res2], axis=1)

In [865]:
final=pd.concat([final, data_s], axis=1)

In [866]:
#Test on one value
testimonial = TextBlob(data_t.values[2])
testimonial.sentiment.polarity

0.1

In [867]:
final

,message,polarity,subjectivity,sentiment
0,#llbite ankylosing spondylitis - have (in date...,0.016667,0.263333,2
1,#as 3 things people with ankylosing spondyliti...,0.000000,0.000000,3
2,working group on the assessment of toxic chemi...,0.100000,0.200000,3
3,i dont know if intense text sex the medication...,0.150000,0.600000,1
4,#imaginedragons singer opens up about his str...,0.000000,0.000000,2
5,chinese medicine cure ankylosing spondylitis c...,-0.250000,0.500000,3
6,pain relief for patients with rheumatoid arthr...,0.000000,0.000000,3
7,suffering from #ankylosingspondylitis? check o...,0.000000,0.000000,3
8,herself goodwill newsletter significant other ...,0.125000,0.625000,2
9,how they did it: 5 people with ankylosing spon...,0.000000,0.000000,3


In [785]:
res6= pd.DataFrame( index=range(0,len(data_t)),columns = {'SentimentPat'} )

In [869]:
#Creating a clumn with clear sentiment
res6.SentimentPat[(final['polarity']>0)]='pos'
res6.SentimentPat[(final['polarity']<0)]='neg'
res6.SentimentPat[(final['polarity']==0)]='neu'

In [870]:
final=pd.concat([final, res6], axis=1)

In [871]:
from sklearn.metrics import confusion_matrix

#final.sentiment[which(final.sentiment=="1")]<-"neg"


In [872]:
final

,message,polarity,subjectivity,sentiment,SentimentPat
0,#llbite ankylosing spondylitis - have (in date...,0.016667,0.263333,2,pos
1,#as 3 things people with ankylosing spondyliti...,0.000000,0.000000,3,neu
2,working group on the assessment of toxic chemi...,0.100000,0.200000,3,pos
3,i dont know if intense text sex the medication...,0.150000,0.600000,1,pos
4,#imaginedragons singer opens up about his str...,0.000000,0.000000,2,neu
5,chinese medicine cure ankylosing spondylitis c...,-0.250000,0.500000,3,neg
6,pain relief for patients with rheumatoid arthr...,0.000000,0.000000,3,neu
7,suffering from #ankylosingspondylitis? check o...,0.000000,0.000000,3,neu
8,herself goodwill newsletter significant other ...,0.125000,0.625000,2,pos
9,how they did it: 5 people with ankylosing spon...,0.000000,0.000000,3,neu


Applying second method from Textblob NB

In [873]:
from textblob.sentiments import NaiveBayesAnalyzer

In [874]:
from textblob import Blobber
tb = Blobber(analyzer=NaiveBayesAnalyzer())

print tb("sentence you want to test").sentiment.classification

neg


In [875]:
tb(data_t[3]).sentiment.p_pos

0.14296541408658425

In [880]:
#Creating a column for results
res4= pd.DataFrame( index=range(0,len(data_t)),columns = {'sentimentNB','pposNB','pnegNB'} )

In [881]:
res4

,pposNB,sentimentNB,pnegNB
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,NaN,NaN,NaN
4,NaN,NaN,NaN
5,NaN,NaN,NaN
6,NaN,NaN,NaN
7,NaN,NaN,NaN
8,NaN,NaN,NaN
9,NaN,NaN,NaN


In [882]:
for i in range(len(data_t)):
    res4['sentimentNB'][i]= tb(data_t[i]).sentiment.classification
    res4['pposNB'].values[i]= tb(data_t[i]).sentiment.p_pos
    res4['pnegNB'].values[i]= tb(data_t[i]).sentiment.p_neg

In [883]:
res4

,pposNB,sentimentNB,pnegNB
0,0.324156,neg,0.675844
1,0.56688,pos,0.43312
2,0.0838154,neg,0.916185
3,0.142965,neg,0.857035
4,0.787871,pos,0.212129
5,0.537264,pos,0.462736
6,0.357929,neg,0.642071
7,0.492796,neg,0.507204
8,0.796022,pos,0.203978
9,0.60678,pos,0.39322


In [884]:
final=pd.concat([final, res4], axis=2)

In [885]:
final

,message,polarity,subjectivity,sentiment,SentimentPat,pposNB,sentimentNB,pnegNB
0,#llbite ankylosing spondylitis - have (in date...,0.016667,0.263333,2,pos,0.324156,neg,0.675844
1,#as 3 things people with ankylosing spondyliti...,0.000000,0.000000,3,neu,0.56688,pos,0.43312
2,working group on the assessment of toxic chemi...,0.100000,0.200000,3,pos,0.0838154,neg,0.916185
3,i dont know if intense text sex the medication...,0.150000,0.600000,1,pos,0.142965,neg,0.857035
4,#imaginedragons singer opens up about his str...,0.000000,0.000000,2,neu,0.787871,pos,0.212129
5,chinese medicine cure ankylosing spondylitis c...,-0.250000,0.500000,3,neg,0.537264,pos,0.462736
6,pain relief for patients with rheumatoid arthr...,0.000000,0.000000,3,neu,0.357929,neg,0.642071
7,suffering from #ankylosingspondylitis? check o...,0.000000,0.000000,3,neu,0.492796,neg,0.507204
8,herself goodwill newsletter significant other ...,0.125000,0.625000,2,pos,0.796022,pos,0.203978
9,how they did it: 5 people with ankylosing spon...,0.000000,0.000000,3,neu,0.60678,pos,0.39322


In [886]:
final.sentimentNB[(final['pposNB']<0.7) &(final['pposNB']>0.3) & (final['pnegNB']<0.7) & (final['pnegNB']>0.3)]='neu'

/Users/imacair/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [887]:
res5= pd.DataFrame( index=range(0,len(data_t)),columns = {'new_sent'} )
#final.sentiment[1][(final['sentiment']==u'2') &(final['sentiment']==u'1')] = 'neg'
#final=pd.concat([final, res5], axis=2)

In [888]:
final.sentiment[0]


'2'

In [889]:
res5[(final.sentiment==u'2')] = 'neg'
res5[(final.sentiment==u'1')] = 'neg'
res5[(final['sentiment']==u'3')] = 'pos'
res5[(final['sentiment']==u'4')] = 'pos'
res5[(final['sentiment']==u'N')] = 'neu'
final=pd.concat([final, res5], axis=2)
#res5

In [890]:
final

,message,polarity,subjectivity,sentiment,SentimentPat,pposNB,sentimentNB,pnegNB,new_sent
0,#llbite ankylosing spondylitis - have (in date...,0.016667,0.263333,2,pos,0.324156,neu,0.675844,neg
1,#as 3 things people with ankylosing spondyliti...,0.000000,0.000000,3,neu,0.56688,neu,0.43312,pos
2,working group on the assessment of toxic chemi...,0.100000,0.200000,3,pos,0.0838154,neg,0.916185,pos
3,i dont know if intense text sex the medication...,0.150000,0.600000,1,pos,0.142965,neg,0.857035,neg
4,#imaginedragons singer opens up about his str...,0.000000,0.000000,2,neu,0.787871,pos,0.212129,neg
5,chinese medicine cure ankylosing spondylitis c...,-0.250000,0.500000,3,neg,0.537264,neu,0.462736,pos
6,pain relief for patients with rheumatoid arthr...,0.000000,0.000000,3,neu,0.357929,neu,0.642071,pos
7,suffering from #ankylosingspondylitis? check o...,0.000000,0.000000,3,neu,0.492796,neu,0.507204,pos
8,herself goodwill newsletter significant other ...,0.125000,0.625000,2,pos,0.796022,pos,0.203978,neg
9,how they did it: 5 people with ankylosing spon...,0.000000,0.000000,3,neu,0.60678,neu,0.39322,pos


In [891]:
from sklearn.metrics import confusion_matrix

Creating confusion matrix

In [894]:
print(confusion_matrix(final.new_sent, final.sentimentNB).transpose())

[[ 75  69  39]
 [117 102 158]
 [133  97 175]]


In [895]:
print(confusion_matrix(final.new_sent, final.SentimentPat).transpose())

[[ 78  29  28]
 [119 121 114]
 [128 118 230]]


In [732]:
print(len(final[(final['sentimentNB']=='neg') & (final['new_sent']=='pos')]))
##print(len(final[(final['sentimentNB']=='pos') & (final['sentiment']<u'3')]))
#print(len(final[(final['sentimentNB']=='neg') & (final['sentiment']>u'2')]))
#print(len(final[(final['sentimentNB']=='neg') & (final['sentiment']<u'3')])) 

39
